In [2]:
import pandas as pd
import numpy as np

In [3]:
data=pd.read_excel(r"G:\dbquery-optimizer\tpcds\notebooks\Query_2_Plan_stats.xls")
data=data.iloc[:,1:]
data

,Query_No,No Plan,Default Plan,Plan 1,Plan 2,Plan 3,mean_duration,min_duration,max_duration,std deviation
0,2,1.00,19.25,23.75,15.25,12.25,14.30,1.00,23.75,8.599419
1,4,8.25,19.50,19.50,19.50,23.50,18.05,8.25,23.50,5.745651
2,5,1.00,15.50,19.25,11.50,15.25,12.50,1.00,19.25,6.988830
3,7,347.25,19.50,54.75,16.00,15.50,90.60,15.50,347.25,144.408124
4,11,15.50,15.50,15.75,11.50,12.25,14.10,11.50,15.75,2.050914
5,12,15.75,11.50,15.25,16.00,12.25,14.15,11.50,16.00,2.110983
6,13,12.00,12.00,15.50,8.50,4.50,10.50,4.50,15.50,4.168333
7,15,12.25,12.00,15.25,11.50,8.00,11.80,8.00,15.25,2.582392
8,17,16.00,43.00,12.25,16.00,16.00,20.65,12.25,43.00,12.599107
9,18,8.50,4.75,19.75,1.00,8.25,8.45,1.00,19.75,7.016498


In [4]:
def extract_parameters(query):
    params=[]
    s=query
    s_index=query.find("item.i_manufact_id = ")+len("item.i_manufact_id = ")
    param1=''
    while(s[s_index]!="\n"):
        param1+=s[s_index]
        s_index+=1
    params.append(param1)
    s_index=query.find("dt.d_moy = ")+len("dt.d_moy = ")
    param2=''
    while(s[s_index]!="\n"):
        param2+=s[s_index]
        s_index+=1
    params.append(param2)
    return params

In [6]:
n_data=[]
labels=[]
for i in range(data.shape[0]-1):
    f=open(r"G:\dbquery-optimizer\tpcds\query_variants\query_2\q_two_"+str(data['Query_No'][i])+".sql",'r')
    qry=f.read()
    params=extract_parameters(qry)
    t=data['min_duration'][i]
    best_plan=""
    if(t==data['No Plan'][i]):best_plan+="No Plan"
    elif(t==data['Default Plan'][i]):best_plan+="Default Plan"
    elif(t==data['Plan 1'][i]):best_plan+="Plan 1"
    elif(t==data['Plan 2'][i]):best_plan+="Plan 2"
    elif(t==data['Plan 3'][i]):best_plan+="Plan 3"
    labels.append(best_plan)
    n_data.append(params)

n_data=np.array(n_data)
labels=np.array(labels)
df=pd.DataFrame({'Parameter 1':n_data[:,0],'Parameter 2':n_data[:,1],'Best Plan':labels})
df['Parameter 1']=pd.to_numeric(df['Parameter 1'])
df['Parameter 2']=pd.to_numeric(df['Parameter 2'])
df

,Parameter 1,Parameter 2,Best Plan
0,17,7,No Plan
1,501,12,No Plan
2,145,9,No Plan
3,665,12,Plan 3
4,462,2,Plan 2
5,128,10,Default Plan
6,71,6,Plan 3
7,255,7,Plan 3
8,34,10,Plan 1
9,615,1,Plan 2


In [7]:
from imblearn.over_sampling import RandomOverSampler
oversample=RandomOverSampler(random_state=42)
X=df.iloc[:,:-1]
y=df['Best Plan']
X=np.array(X)
y=np.array(y)
X,y=oversample.fit_sample(X,y)
print(X.shape,y.shape)

(60, 2) (60,)


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators=100,criterion='gini')
clf.fit(X_train,y_train)

RandomForestClassifier()

In [13]:
y_pred=clf.predict(X_test)
y_tpred=clf.predict(X_train)
print(y_pred, y_test)
print("Test accuracy:",accuracy_score(y_test,y_pred))
print("Train accuracy:",accuracy_score(y_train,y_tpred))

['Plan 3' 'Plan 2' 'Plan 2' 'Plan 2' 'Plan 1' 'Plan 1' 'Default Plan'
 'Plan 2' 'Default Plan' 'Plan 2' 'No Plan' 'Plan 2'] ['Plan 3' 'Plan 2' 'Plan 2' 'Plan 2' 'Plan 1' 'Plan 1' 'Default Plan'
 'No Plan' 'Default Plan' 'Plan 2' 'No Plan' 'Plan 2']
Test accuracy: 0.9166666666666666
Train accuracy: 1.0


In [14]:
test=np.array(df.iloc[:,:-1])
res=clf.predict(test)
tab=[]
for i in range(len(res)):
    row=[]
    row.append(data['Query_No'][i])
    ac_time=data['No Plan'][i]
    im_time=data[res[i]][i]
    im_percent=((ac_time-im_time)/ac_time)*100
    row.append(ac_time)
    row.append(im_time)
    row.append(im_percent)
    tab.append(row)

tab=np.array(tab)
tab=pd.DataFrame({'Query No':tab[:,0],'Default_Execution_Time':tab[:,1],'Improved_Execution_Time':tab[:,2],'Percentage_Improvement':tab[:,3]})
tab=tab.astype({'Query No':int})
mean=pd.DataFrame({'Query No':['Mean'],'Default_Execution_Time':[tab['Default_Execution_Time'].mean()],'Improved_Execution_Time':[tab['Improved_Execution_Time'].mean()],'Percentage_Improvement':[tab['Percentage_Improvement'].mean()]})
tab=tab.append(mean,ignore_index=False,sort=False)
tab.to_excel('Query_2_Improvement.xls')
tab

,Query No,Default_Execution_Time,Improved_Execution_Time,Percentage_Improvement
0,2,1.000000,1.000000,0.000000
1,4,8.250000,8.250000,0.000000
2,5,1.000000,1.000000,0.000000
3,7,347.250000,15.500000,95.536357
4,11,15.500000,11.500000,25.806452
5,12,15.750000,11.500000,26.984127
6,13,12.000000,4.500000,62.500000
7,15,12.250000,8.000000,34.693878
8,17,16.000000,12.250000,23.437500
9,18,8.500000,1.000000,88.235294


In [15]:
from sklearn.tree import DecisionTreeClassifier
clf=DecisionTreeClassifier()
clf.fit(X_train,y_train)

DecisionTreeClassifier()

In [16]:
y_pred=clf.predict(X_test)
y_tpred=clf.predict(X_train)
print(y_pred, y_test)
print("Test accuracy:",accuracy_score(y_test,y_pred))
print("Train accuracy:",accuracy_score(y_train,y_tpred))

['Plan 3' 'Plan 2' 'Plan 2' 'Plan 2' 'Plan 1' 'Plan 1' 'Default Plan'
 'Plan 2' 'Default Plan' 'Plan 2' 'No Plan' 'Plan 2'] ['Plan 3' 'Plan 2' 'Plan 2' 'Plan 2' 'Plan 1' 'Plan 1' 'Default Plan'
 'No Plan' 'Default Plan' 'Plan 2' 'No Plan' 'Plan 2']
Test accuracy: 0.9166666666666666
Train accuracy: 1.0
